In [12]:
# Блок 1: Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, ShuffleSplit
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler

In [13]:
# Блок 2: Загрузка данных
data = pd.read_excel('yn.xlsx')
df = pd.DataFrame(data)

In [14]:
# Блок 3: Определение входных и выходных данных
X = df[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']]
y = df['yn']

In [15]:
# Блок 4: Разделение данных на обучающую, тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [16]:
# Нормализация данных: приведение всех признаков к одному масштабу
from sklearn.compose import ColumnTransformer

# Нормализация данных: приведение всех признаков к одному масштабу
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Обучение и преобразование обучающего набора данных
X_test_scaled = scaler.transform(X_test)  # Преобразование тестового набора данных

In [17]:
# Блок 5: Сетка гиперпараметров и кросс-валидация
# Задание сетки гиперпараметров для модели MLPRegressor
# Определение параметров для поиска по сетке
# Уточненная сетка параметров
param_grid = {
    'activation': ['logistic'],  
    'solver': ['adam'],
    'alpha': [0.97, 0.98, 0.99],  # Вариации вокруг 0.46
    'beta_1': [0.8, 0.9, 0.995],  # Разные значения beta_1
    'beta_2': [0.9, 0.98],  # Варианты beta_2
    'early_stopping': [False],  # Проверка влияния ранней остановки
    'epsilon': [1e-10],  # Вариации epsilon
    'hidden_layer_sizes': [(11,), (12,), (13,)],  # Варианты числа нейронов
    'learning_rate_init': [0.01, 0.03, 0.05],  # Разные темпы обучения
    'n_iter_no_change': [10],  # Проверка устойчивости к изменениям
    'tol': [0.001, 0.01, 0.1],  # Влияние чувствительности к ошибке
    'random_state': [74]  # Фиксированный сид для воспроизводимости
}

# Настройка кросс-валидации
cv = ShuffleSplit(
    n_splits=5,         # количество фолдов
    test_size=0.25,       # перемешивание данных
    random_state=73     # для воспроизводимости результатов
)

grid_search = GridSearchCV(
    estimator=MLPRegressor(random_state=42), 
    param_grid=param_grid, 
    cv=cv,  
    scoring='neg_mean_absolute_percentage_error',  
    n_jobs=-1,  
    verbose=2  
)


In [18]:
# Блок 6: Обучение GridSearchCV
print("Начинается подбор гиперпараметров...")
grid_search.fit(X_train_scaled, y_train)  # Обучение модели с подбором гиперпараметров

Начинается подбор гиперпараметров...
Fitting 5 folds for each of 486 candidates, totalling 2430 fits


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=73, test_size=0.25, train_size=None),
             estimator=MLPRegressor(random_state=42), n_jobs=-1,
             param_grid={'activation': ['logistic'],
                         'alpha': [0.97, 0.98, 0.99],
                         'beta_1': [0.8, 0.9, 0.995], 'beta_2': [0.9, 0.98],
                         'early_stopping': [False], 'epsilon': [1e-10],
                         'hidden_layer_sizes': [(11,), (12,), (13,)],
                         'learning_rate_init': [0.01, 0.03, 0.05],
                         'n_iter_no_change': [10], 'random_state': [74],
                         'solver': ['adam'], 'tol': [0.001, 0.01, 0.1]},
             scoring='neg_mean_absolute_percentage_error', verbose=2)

In [19]:
# Блок 7: Лучшая модель
best_params = grid_search.best_params_  # Получение лучших гиперпараметров
print("Лучшие параметры:", grid_search.best_params_)
print("Лучший результат:", -grid_search.best_score_ * 100, "%")

# Получение модели с лучшими параметрами
best_model = grid_search.best_estimator_

# Прогнозирование на тестовом наборе данных с лучшей моделью
y_pred_best = best_model.predict(X_test_scaled)

Лучшие параметры: {'activation': 'logistic', 'alpha': 0.98, 'beta_1': 0.9, 'beta_2': 0.98, 'early_stopping': False, 'epsilon': 1e-10, 'hidden_layer_sizes': (12,), 'learning_rate_init': 0.03, 'n_iter_no_change': 10, 'random_state': 74, 'solver': 'adam', 'tol': 0.1}
Лучший результат: 9.483148063847095 %


In [20]:
# Блок 8: Вычисление метрик
# Вычисление MAPE (Mean Absolute Percentage Error) для оценки точности
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100
# Вычисление RMSE (Root Mean Squared Error) для оценки отклонений предсказаний от реальных значений
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

# Вывод лучших метрик
print(f"MAPE: {mape_best:.2f}%")
print(f"RMSE: {rmse_best:.2f}")

MAPE: 9.39%
RMSE: 2.72


In [21]:
# Блок 9: Создание DataFrame для тестовых значений
# Сравнение истинных значений с предсказанными для тестового набора
test_comparison_model = pd.DataFrame({"yn": y_test.values, "y_pred_best": y_pred_best})
print(test_comparison_model)

    yn  y_pred_best
0   29    27.782710
1   25    27.385550
2   28    26.315870
3   25    24.858327
4   22    24.406642
5   22    25.428162
6   23    27.285000
7   28    23.025363
8   29    28.638285
9   30    27.577538
10  28    28.323030
11  23    26.487988
12  26    22.638744
13  25    27.751807
14  26    25.686349
15  26    23.753039
16  20    24.391314
17  25    23.909303


In [22]:
# Блок 10: Сохранение результатов
# Сохранение результатов и метрик в Excel файл
test_comparison_best = pd.DataFrame({"yn": y_test.values, "y_pred_py": y_pred_best})
metrics_best = pd.DataFrame({"Метрика": ["MAPE", "RMSE"], "Значение": [mape_best, rmse_best]})

# Запись в Excel
output_file = 'models_results_optimized.xlsx'  # Имя выходного файла
with pd.ExcelWriter(output_file) as writer:
    test_comparison_best.to_excel(writer, sheet_name='Результаты лучшей модели', index=False)  # Результаты модели
    metrics_best.to_excel(writer, sheet_name='Метрики лучшей модели', index=False)  # Метрики модели